---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [65]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [66]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', \
          'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', \
          'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', \
          'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', \
          'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', \
          'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', \
          'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', \
          'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', \
          'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', \
          'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', \
          'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', \
          'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', \
          'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [67]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    import pandas as pd
    import numpy as np
    filename = 'university_towns.txt'
    with open(filename, 'r') as myfile:
        data = myfile.read().split('\n') 
        
    state_iii = []
    for pos, entry in enumerate(data):
        if entry.replace('[edit]', '') in list(states.values()):
            # print(entry.replace('[edit]', ''))
            state_iii.append(pos)
    # print(len(state_iii))

    # State_names = []
    bu = []
    for idx in range(len(state_iii)-1): 
        State = data[state_iii[idx]]
        State = State[:State.find('[')]

        RegionName = data[state_iii[idx]+1: state_iii[idx+1]]
        for region in RegionName:
            region = region[:region.find('(')-1]
            # print(region)
            bu.append([State, region])
    # print(bu)
    
    df = pd.DataFrame(bu, columns=['State', 'RegionName'])
    # print(df.head())
    
    return df  # df

get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [68]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
   
    # GDP over time of the United States in current dollars 
    # (use the chained value in 2009 dollars), in quarterly intervals
    # For this assignment, only look at GDP data from the first quarter of 2000 onward.
    
    fname = 'gdplev.xls'
    df = pd.read_excel(fname, skiprows=7, usecols=(4, 6), names=['YearQ', 
                                                                 'GDP [Billion Dollars]']) 
    iii = int(np.squeeze(np.where(df['YearQ'] == '2000q1')))
    df = df.iloc[iii:]
    
    found = 0
    df['decline'] = df['GDP [Billion Dollars]'].diff() < 0    
    # print(df.head(45))    
    # find quarter of recession start
    # A recession is defined as starting with two consecutive quarters of GDP decline, 
    # and ending with two consecutive quarters of GDP growth
    while not found: 
        for ii in range(len(df)-1): 
            # print(df['decline'].iloc[ii])
            if df['decline'].iloc[ii] and df['decline'].iloc[ii+1]:
                # print(df['YearQ'].iloc[ii])
                year = df['YearQ'].iloc[ii]

                # check if there's a recession, by checking if there's consecutive gain after this period
                to_check = df.iloc[ii+2: len(df)]
                # print(to_check.head())
                gain = np.array(to_check['GDP [Billion Dollars]'].diff() > 0) 
                
                for jj in range(len(gain)-1):
                    if gain[jj] and gain[jj+1]:
                        found = 1
                        return year


get_recession_start()

'2008q3'

In [69]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    fname = 'gdplev.xls'
    df = pd.read_excel(fname, skiprows=7, usecols=(4, 6), names=['YearQ', 
                                                                 'GDP [Billion Dollars]']) 
    
    start_q = get_recession_start()
    iii = int(np.squeeze(np.where(df['YearQ'] == start_q)))
    df = df.iloc[iii:]
    # print(df.head())
    
    found = 0
    df['gain'] = df['GDP [Billion Dollars]'].diff() > 0    
    # find quarter of recession end
    
    while not found: 
        for ii in range(len(df)-1): 
            if df['gain'].iloc[ii] and df['gain'].iloc[ii+1]:
                year = df['YearQ'].iloc[ii+1]
                found=1
                break

    return year

get_recession_end()

'2009q4'

In [70]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    fname = 'gdplev.xls'
    df = pd.read_excel(fname, skiprows=7, usecols=(4, 6), names=['YearQ', 
                                                                 'GDP [Billion Dollars]']) 
    
    start_q = get_recession_start()
    end_q = get_recession_end()
    
    iii = int(np.squeeze(np.where(df['YearQ'] == start_q)))
    jjj = int(np.squeeze(np.where(df['YearQ'] == end_q)))
    df = df.iloc[iii:jjj+1]
    
    # print(df.head())
    
    found = 0
    df['gain'] = df['GDP [Billion Dollars]'].diff() > 0    
    for ii in range(len(df)-1):
        if (df['gain'].iloc[ii] == False) and (df['gain'].iloc[ii+1] == True):
            return df['YearQ'].iloc[ii]
    

get_recession_bottom()

'2009q2'

In [75]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    fname = 'City_Zhvi_AllHomes.csv'
    df = pd.read_csv(fname)
    # print(df.head())    
    
    year_list = list(map(str, range(2000, 2017)))
    # print(year_list)
    target_colname = []

    cols = []
    for year in year_list:
        for i in range(1, 13):
            cols.append(year + '-' + str(i).zfill(2))
            
    cols.remove('2016-09')
    cols.remove('2016-10')
    cols.remove('2016-11')
    cols.remove('2016-12')
        
    ddf = df[cols]
    # print(ddf.head())
        
    res = ddf.groupby(pd.PeriodIndex(ddf.columns, freq='Q'), axis=1).mean().rename(columns=lambda c: str(c).lower())
    # print(res.head())
    
    res['State'] = df['State']
    res["RegionName"] = df["RegionName"]
    # print(res.head())
    
    # should have a multi-index in the shape of ["State","RegionName"]
    res.set_index(["State","RegionName"], inplace=True)

    return res

convert_housing_data_to_quarters()
# print(convert_housing_data_to_quarters().shape)

          2000q1         2000q2         2000q3         2000q4         2001q1  \
0            NaN            NaN            NaN            NaN            NaN   
1  207066.666667  214466.666667  220966.666667  226166.666667  233000.000000   
2  138400.000000  143633.333333  147866.666667  152133.333333  156933.333333   
3   53000.000000   53633.333333   54133.333333   54700.000000   55333.333333   
4  111833.333333  114366.666667  116000.000000  117400.000000  119600.000000   

          2001q2         2001q3         2001q4         2002q1         2002q2  \
0            NaN            NaN            NaN            NaN            NaN   
1  239100.000000  245066.666667  253033.333333  261966.666667  272700.000000   
2  161800.000000  166400.000000  170433.333333  175500.000000  177566.666667   
3   55533.333333   56266.666667   57533.333333   59133.333333   60733.333333   
4  121566.666667  122700.000000  124300.000000  126533.333333  128366.666667   

       ...              2014q4        

In [76]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).
    
    Hypothesis: University towns have their mean housing prices less effected by recessions. 
    # (there's a difference btw two groups)
    
    Run a t-test to compare the ratio of the mean price of houses in university towns 
    the quarter before the recession starts compared to the recession bottom. 
    (price_ratio=quarter_before_recession/recession_bottom)
    
    '''
        
    startq = get_recession_start()
    bottomq = get_recession_bottom()
    
    dff = convert_housing_data_to_quarters()
    df = dff[[startq, bottomq]]
    df = df.reset_index()
    df['price_ratio'] = df[startq]/df[bottomq]
    
    blah = get_list_of_university_towns()
    # blah = blah.set_index(['State',  'RegionName'])
    
    # Use this dictionary to map state names to two letter acronyms
    states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', \
              'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', \
              'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', \
              'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', \
              'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', \
              'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', \
              'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', \
              'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', \
              'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', \
              'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', \
              'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', \
              'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', \
              'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
    df = df.replace({"State": states})
    # print(df.head())              
        
    df = df.set_index(['State', 'RegionName'])
    blah = blah.set_index(['State', 'RegionName'])
                          
    # print(df.index.isin(blah.index))
    uni = df[df.index.isin(blah.index)]['price_ratio'].dropna()
    # print(uni.head())
    
    nonuni = df[~df.index.isin(blah.index)]['price_ratio'].dropna()
    # print(nonuni.head())
    
    from scipy import stats
    bu = stats.ttest_ind(uni, nonuni)
    # print(bu)
    
    # whether the alternative hypothesis sis true or not as well as the p-value of the confidence. 
    
    threshold = 0.01
    if bu[1] < 0.01:
        # reject H0
        different = True
    else: 
        different = False
        # cannot reject

    # The value for better should be either "university town" or "non-university town"
    # depending on which has a lower mean price ratio (which is equivilent to a
    # reduced market loss).

    uni = uni.reset_index()
    uni.columns=['State', 'RegionName', 'Ratio']
    nonuni = nonuni.reset_index()
    nonuni.columns=['State', 'RegionName', 'Ratio']
    if uni['Ratio'].mean() < nonuni['Ratio'].mean():
        better = 'university town'
    else:
        better = 'non-university town'
        
    
    return (different, bu[1], better)

run_ttest()


          2000q1         2000q2         2000q3         2000q4         2001q1  \
0            NaN            NaN            NaN            NaN            NaN   
1  207066.666667  214466.666667  220966.666667  226166.666667  233000.000000   
2  138400.000000  143633.333333  147866.666667  152133.333333  156933.333333   
3   53000.000000   53633.333333   54133.333333   54700.000000   55333.333333   
4  111833.333333  114366.666667  116000.000000  117400.000000  119600.000000   

          2001q2         2001q3         2001q4         2002q1         2002q2  \
0            NaN            NaN            NaN            NaN            NaN   
1  239100.000000  245066.666667  253033.333333  261966.666667  272700.000000   
2  161800.000000  166400.000000  170433.333333  175500.000000  177566.666667   
3   55533.333333   56266.666667   57533.333333   59133.333333   60733.333333   
4  121566.666667  122700.000000  124300.000000  126533.333333  128366.666667   

       ...              2014q4        

(True, 0.0054964273536938875, 'university town')